In [0]:
#import shutil

#shutil.rmtree('./flower_data')


In [57]:
# Installing Required Packages

'''
!pip install torch
!pip install torchvision
!pip install requests
'''

'\n!pip install torch\n!pip install torchvision\n!pip install requests\n'

In [0]:
# Importing Packages

import os
import torch
import torchvision
from torchvision import transforms, datasets
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


In [0]:
# Data Transformation

data_transform = transforms.Compose([
    #transforms.RandomResizedCrop(224),
    transforms.CenterCrop(500),
    #transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.4406], std=[0.229,0.224,0.225])
])

data_transform

Compose(
    CenterCrop(size=(500, 500))
    Resize(size=224, interpolation=PIL.Image.BILINEAR)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.4406], std=[0.229, 0.224, 0.225])
)

In [0]:
# Downloading Data Set

import requests, zipfile, io

r = requests.get('https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

print('Done')

Done


In [0]:
# Loading images and creating labels

data_dir = './flower_data/'
train_dir = os.path.join(data_dir, 'train/') 
test_dir = os.path.join(data_dir, 'valid/') 

train_data = datasets.ImageFolder(train_dir, transform = data_transform) 
test_data = datasets.ImageFolder(test_dir, transform = data_transform) 


In [22]:
print('No of training images', len(train_data))
print('No of testing images', len(test_data))

No of training images 6552
No of testing images 818


In [0]:
# Load the data

train_loader = torch.utils.data.DataLoader(train_data, batch_size=200, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=200, shuffle=True)

In [0]:
# Load pre-train model

model = torchvision.models.resnet18(pretrained=True)
#model = torchvision.models.resnet152(pretrained=True)

# Reset Gradient

for param in model.parameters():
  param.requires_grad = False  

In [36]:

# No.of neurons in last layers

num_features = model.fc.in_features
num_features


512

In [0]:
# Building Model Architecture

class PSC(nn.Module):
  def __init__(self):
    super(PSC, self).__init__()
    self.fc1 = nn.Linear(in_features = num_features, out_features=256)
    self.dropout = nn.Dropout(0.25)
    self.fc2 = nn.Linear(in_features = 256, out_features=64)
    self.fc3 = nn.Linear(in_features = 64, out_features=256)
    self.fc4 = nn.Linear(in_features = 256, out_features=102)
    
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x
    

In [0]:
model.fc = PSC()

#model.load_state_dict(torch.load('pr_model.pt'), strict=False) #SFSAver10
torch.save(model.state_dict(), 'pr_model.pt')
#torch.save({'state_dict': model.state_dict()}, 'checkpoint.pth.tar')

In [0]:
# Define Loss and Optimizer

loss_fn = torch.nn.CrossEntropyLoss()

#loss_fn = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters())


In [58]:
'''
!pip install Pillow==4.1.1
!pip install PIL
!pip install image

from PIL import *

'''

'\n!pip install Pillow==4.1.1\n!pip install PIL\n!pip install image\n\nfrom PIL import *\n\n'

In [42]:
# Verify GPU availability

torch.cuda.is_available()

True

In [43]:
# Training Final Model

n_epochs=30
accuracy_val = 0.0

for epoch in range(1, n_epochs+1):
  train_loss = 0.0
  
  for batch_i, (data, target) in enumerate(train_loader):
    
    if torch.cuda.is_available():
      data, target, model = data.cuda(), target.cuda(), model.cuda()
              
    output = model(data)
    loss = loss_fn(output, target)
    train_loss +=loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()  
   
    
    if batch_i % 20 == 15:
      print("Epoch: {}  Batch: {} loss: {}".format(epoch, batch_i, train_loss/20))
      
      train_loss = 0.0
      model.eval()
      total_correct = 0
      total = 0
      max_avg_output = torch.argmax(output, dim=1)
      
      for number in range(len(data)):
        correct = max_avg_output[number] == target[number]
        total_correct += torch.Tensor.cpu(correct).numpy()
        total += 1
        
      print('Correct: ', total_correct)
      print('Total: ', total)
      acc = (total_correct/total)
      print('Training Accuracy: ', acc)
      
      if acc > accuracy_val:
        torch.save({'state_dict': model.state_dict()}, 'checkpoint.pth.tar')
        accuracy_val = acc
        
      model.train()
      


Epoch: 1  Batch: 15 loss: 3.6036813497543334
Correct:  22
Total:  200
Training Accuracy:  0.11
Epoch: 2  Batch: 15 loss: 2.4376044631004334
Correct:  62
Total:  200
Training Accuracy:  0.31
Epoch: 3  Batch: 15 loss: 1.4426110506057739
Correct:  118
Total:  200
Training Accuracy:  0.59
Epoch: 4  Batch: 15 loss: 1.0304762601852417
Correct:  141
Total:  200
Training Accuracy:  0.705
Epoch: 5  Batch: 15 loss: 0.8245095998048783
Correct:  151
Total:  200
Training Accuracy:  0.755
Epoch: 6  Batch: 15 loss: 0.6871590554714203
Correct:  144
Total:  200
Training Accuracy:  0.72
Epoch: 7  Batch: 15 loss: 0.581581249833107
Correct:  163
Total:  200
Training Accuracy:  0.815
Epoch: 8  Batch: 15 loss: 0.49911018311977384
Correct:  167
Total:  200
Training Accuracy:  0.835
Epoch: 9  Batch: 15 loss: 0.47403434216976165
Correct:  174
Total:  200
Training Accuracy:  0.87
Epoch: 10  Batch: 15 loss: 0.41163163632154465
Correct:  173
Total:  200
Training Accuracy:  0.865
Epoch: 11  Batch: 15 loss: 0.36254

In [0]:
torch.save({'state_dict': model.state_dict()}, 'checkpoint_final.pth.tar')

In [0]:
torch.save({'state_dict': model.state_dict()}, 'checkpoint_final.pt')

In [0]:
torch.save(model.state_dict(), 'checkpoint_final_v1.pt')

In [52]:
model

PSC(
  (fc1): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.25)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=102, bias=True)
)

In [0]:
# Final Submission

In [0]:


class PSC(nn.Module):    
      def __init__(self):            
            super().__init__()
            self.fc1 = nn.Linear(in_features=512, out_features=256)
            self.dropout = nn.Dropout(0.25)
            self.fc2 = nn.Linear(in_features = 256, out_features=64)
            self.fc3 = nn.Linear(in_features = 64, out_features=256)
            self.fc4 = nn.Linear(in_features = 256, out_features=102)
    
      def forward(self, x):
            x = F.relu(self.fc1(x))
            x = self.dropout(x)
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = self.fc4(x)
            return x


model = torchvision.models.resnet18(pretrained=True)


model.fc = PSC()

checkpoint = torch.load('checkpoint_final.pt', map_location='cpu')
model.load_state_dict(checkpoint['state_dict'], strict=False)

In [0]:
# Load your model to this variable
#model = torch.load('PyTorch_model.pt', map_location='cpu')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

In [0]:
from collections import namedtuple
from torch import nn


def test_model(model1, criterion=nn.NLLLoss, image_size=224):
    Tester = namedtuple('Tester', ['model1', 'criterion', 'image_size'])
    return Tester(model1, criterion, image_size)
  
test_model(model1, criterion=nn.NLLLoss, image_size=224)